# Домашня робота №3
Будемо використовувати *Збірник задач з теорії ймовірностей та математичної статистики: навч. посібник / В.В. Голомозий, М.В. Карташов, К.В. Ральченко. – К.: Видавничо-поліграфічний центр «Київський університет», 2015. – 366 с.*
Електронну версію збірника можна знайти [тут](http://probability.univ.kiev.ua/userfiles/kmv/gkr-problems.pdf).

## 1 Задача 1.3.12
Знайти ймовiрнiсть того, що серед трьох цифр, кожна з яких вибрана навмання, буде лише 1, 2, 3 рiзних.

Обчисліть теоретичну (повним перебором) та еміричну (симулюванням $100000$ експериментів) імовірності.

In [1]:
from itertools import product
from numpy.random import choice


def task1Empiric():
    probability = [0, 0, 0]
    digits = product(*[range(10)]*3)
    for digit in digits:
        probability[3 - len(set(digit))] += 1
    return list(map(lambda x: x / 10 ** 3, probability))


def task1Theoretic(tests=10000):
    probability = [0, 0, 0]
    for _ in range(tests):
        digit = choice(range(10), size=3)
        probability[3 - len(set(digit))] += 1
    return list(map(lambda x: x / tests, probability))
    

print(f'empiric - {task1Empiric()}')
print(f'theoretic - {task1Theoretic()}')




# 2 Задача 1.3.14
З послiдовностi чисел $1, 2, . . . , n$ вибирають навмання $k$ рiзних чисел.
Яка ймовiрнiсть того, що:
1. кожне з вибраних чисел кратне даному числу $p$;
2. кожне з вибраних чисел кратне хоча б одному з двох взаємно простих чисел $p$ i $q$;
3. серед вибраних чисел є хоча б одне кратне $p$?

Напишіть  відповідні функції для обрахунку теоретичної (повним перебором) та еміричної (симулюванням $100000$ експериментів) імовірностей в залежності від параметрів $n, k, p, q$.
Виведіть результат для
- $n = 25, k = 5, p = 3, q = 4$;
- $n = 25, k = 10, p = 3, q = 4$.

In [2]:
from numpy.random import choice
from itertools import combinations_with_replacement


def gcd(a, b):
    while a != b:
        if a > b:
            a -= b
        else:
            b -= a
    return a


def prime(a, b):
    return gcd(a, b) == 1


def task2Empiric(n, k, p, q, test=10000):
    probability = [0, 0, 0]
    for _ in range(test):
        digits = choice(range(1, n + 1), k)
        success0 = True  # для всех
        success1 = True  # для двух взаимопростых
        success2 = False # для хотя бы одного числа
        
        for digit in digits:
            if digit % p:
                success0 = False
            if not digit % p:
                success2 = True
            if digit % p and digit % q:
                success1 = False
        if success0:
            probability[0] += 1
        if success1:
            probability[1] += 1
        if success2:
            probability[2] += 1
    if not prime(p, q):
        probability[1] = 0
    return list(map(lambda x: x / test, probability))


def task2Theoretic(n, k, p, q):
    
    def check(num):
        
        """ проверка, True если все цифры числа разные"""
        
        return len(set(num)) == len(num)
    
    probability = [0, 0, 0]
    digits = combinations_with_replacement(range(1, n + 1), k)
    c = 0
    for i in digits:
        if not check(i):
            continue
        c += 1
        success0 = True  # для всех
        success1 = True  # для двух взаимопростых
        success2 = False # для хотя бы одного числа
        for digit in i:
            if digit % p:
                success0 = False
            if not digit % p:
                success2 = True
            if digit % p and digit % q:
                success1 = False
        if success0:
            probability[0] += 1
        if success1:
            probability[1] += 1
        if success2:
            probability[2] += 1
    if not prime(p, q):
        probability[1] = 0
    return list(map(lambda x: x / c, probability))
    
    
print(task2Empiric(25, 5, 3, 4))
print(task2Empiric(25, 10, 3, 4))
print(task2Theoretic(25, 5, 3, 4))
print(task2Theoretic(25, 10, 3, 4))

# 3 Задача 1.4.6
В коморi знаходяться $n$ пар черевикiв.
З них випадковим чином вибираються $2k$ черевикiв.
Яка ймовiрнiсть того, що серед вибраних черевикiв:
1. вiдсутнi парнi;
2. є рiвно одна комплектна пара;
3. є двi комплектнi пари?

Напишіть  відповідні функції для обрахунку теоретичної (повним перебором) та еміричної (симулюванням $100000$ експериментів) імовірностей в залежності від параметрів $n, k$.
Виведіть результат для
- $n = 8, k = 4$;
- $n = 13, k = 5$.

In [3]:
from itertools import combinations
from numpy.random import shuffle

alphabet = "abcdefghijklmnopqrstuvwxyz"


def task3Theoretic(n, k):
    cur = alphabet[:n]
    pairs = [(i, 1) for i in cur] + [(i, 2) for i in cur]
    probability = [0, 0, 0]
    c = 0
    for comb in combinations(pairs, 2 * k):
        c += 1
        success1 = True # для парных
        tmp = {}
        for a in cur:
            tmp[a] = 0
        for pair in comb:
            tmp[pair[0]] += 1
            
        num_pairs = len([i for i in tmp.values() if i == 2])
        if num_pairs == 0:
            probability[0] += 1
        if num_pairs == 1:
            probability[1] += 1
        if num_pairs == 2:
            probability[2] += 1
    return list(map(lambda x: x / c, probability))


def task3Emprirc(n, k, test=10000):
    cur = alphabet[:n]
    pairs = [(i, 1) for i in cur] + [(i, 2) for i in cur]
    probability = [0, 0, 0]
    for _ in range(test):
        temp = pairs[:]
        shuffle(temp)
        comb = temp[:2 * k]
        success1 = True # для парных
        tmp = {}
        for a in cur:
            tmp[a] = 0
        for pair in comb:
            tmp[pair[0]] += 1

        num_pairs = len([i for i in tmp.values() if i == 2])
        if num_pairs == 0:
            probability[0] += 1
        if num_pairs == 1:
            probability[1] += 1
        if num_pairs == 2:
            probability[2] += 1
    return list(map(lambda x: x / test, probability))


print(task3Theoretic(8, 4))
print(task3Theoretic(13, 5))

print(task3Emprirc(8, 4))
print(task3Emprirc(13, 5))

# 4 Задача 1.2.15
Нехай $\Omega = {1, 2, \ldots, 2n}$.
Всiм числам приписанi ймовiрностi, пропорцiйнi логарифмам цих чисел.
Знайти цi ймовiрностi.
Знайти ймовiрнiсть того, що в результатi експерименту з’явиться:
1. парне число;
2. непарне число.

Напишіть функцію для обрахунку еміричної (симулюванням $100000$ експериментів) імовірності в залежності від параметра $n$.
Виведіть результат для
- $n = 10$;
- $n = 25$.

In [4]:
from numpy import log
from numpy.random import choice


def count4(n):
    coef = 1 / sum ([log(i) for i in range(1, 2*n+1)])
    prob = [coef * log(i) for i in range(1, 2*n + 1)]
    num = [0, 0]
    
    for _ in range(100000):
        tmp = choice(range(1, 2*n+1), p=prob)
        num[tmp%2] += 1
    return coef, prob, list(map(lambda x: x/100000, num))


n = (10, 25)

for i in n:
    coef, prob, num = count4(i)
    print(f'n = {i}')
    print('=================')
    print(f'probabillties: \n {prob}')
    print('=================')
    print(f'num: {num}')